# Workflow Prescription Agent Comparison

Testing two approaches for prompt engineering:
1. **Single Prompt**: Combined system instructions and examples in one template
2. **Split System/Human**: Separate system prompt and human message templates

This comparison will help identify which approach produces better results for workflow prescription.


In [1]:
# Setup and imports
import os
import sys
from pathlib import Path
from typing import List
from pydantic import BaseModel, Field

# Add project root to path
project_root = Path('.').resolve().parent.parent.parent
sys.path.insert(0, str(project_root))

# Import utilities
sys.path.insert(0, str(project_root / "agents" / "zPrototyping"))
from langgraph_utils import create_langchain_structured_agent, merge_prompt_with_examples

# Schema definition
class WorkflowPrescriptionOutput(BaseModel):
    """Schema for workflow prescription agent output"""
    workflows: List[str] = Field(description="List of recommended workflows")
    reasoning: str = Field(description="Explanation of workflow selection")
    confidence: float = Field(description="Confidence score", ge=0.0, le=1.0)
    priority: str = Field(description="Priority level: low, medium, high")

print("✅ Setup complete")


✅ Setup complete


In [2]:
# Create LLM instance
try:
    from langchain_anthropic import ChatAnthropic
    llm = ChatAnthropic(model="claude-3-5-haiku-latest", temperature=0.1)
    print("✅ Using Claude LLM")
except ImportError:
    llm = None
    print("⚠️ Using mock mode")

# Approach 1: Single Prompt (Original)
print("\n🔧 Creating Single Prompt Agent...")
single_prompt_agent = create_langchain_structured_agent(
    name="SinglePrompt",
    prompt_path="workflow_prescription_prompt.md",
    examples_path="workflow_prescription_examples.json",
    output_schema=WorkflowPrescriptionOutput,
    llm=llm
)

print(f"✅ Single prompt agent: {single_prompt_agent.__name__}")
# Get system message length for single prompt agent
single_prompt = merge_prompt_with_examples(
    prompt_path="workflow_prescription_prompt.md", 
    examples_path="workflow_prescription_examples.json"
)
print("📋 System message length:", len(single_prompt))
# Estimate token costs
def estimate_tokens(text):
    """Rough token estimate based on word count"""
    return len(text.split()) * 1.3  # Rough approximation

def estimate_cost(tokens, model="claude-3-haiku"):
    """Estimate cost in USD based on token count"""
    if model == "claude-3-haiku":
        input_cost = 0.25/1000000  # $0.25/1M input tokens
        output_cost = 1.25/1000000  # $1.25/1M output tokens
        return tokens * input_cost, tokens * output_cost
    return 0, 0

# Calculate costs for single prompt approach
single_tokens = estimate_tokens(single_prompt)
single_input_cost, single_output_cost = estimate_cost(single_tokens)
print(f"\n💰 Single Prompt Cost Estimates:")
print(f"Estimated tokens: {single_tokens:.0f}")
print(f"Input cost: ${single_input_cost:.6f}")
print(f"Output cost: ${single_output_cost:.6f}")


✅ Using Claude LLM

🔧 Creating Single Prompt Agent...
✅ Single prompt agent: SinglePrompt_langchain_agent
📋 System message length: 3205

💰 Single Prompt Cost Estimates:
Estimated tokens: 530
Input cost: $0.000133
Output cost: $0.000663


In [3]:
# Approach 2: Split System/Human Prompts
print("🔧 Creating Split Prompt Agent...")

# Create system message from system prompt + examples
system_prompt = merge_prompt_with_examples(
    prompt_path="workflow_prescription_system.md",
    examples_path="workflow_prescription_examples.json"
)

# Create human message template (will be filled per request)
with open("workflow_prescription_human.md", "r") as f:
    human_template = f.read()

split_prompt_agent = create_langchain_structured_agent(
    name="SplitPrompt", 
    prompt_path="workflow_prescription_system.md",
    examples_path="workflow_prescription_examples.json",
    output_schema=WorkflowPrescriptionOutput,
    system_message=system_prompt,  # Use custom system message
    llm=llm
)

print(f"✅ Split prompt agent: {split_prompt_agent.__name__}")
print("📋 System message length:", len(system_prompt))
print("📋 Human template length:", len(human_template))

# Calculate costs for split prompt approach
system_tokens = estimate_tokens(system_prompt)
human_tokens = estimate_tokens(human_template)
split_total_tokens = system_tokens + human_tokens
split_input_cost, split_output_cost = estimate_cost(split_total_tokens)

print(f"\n💰 Split Prompt Cost Estimates:")
print(f"System tokens: {system_tokens:.0f}")
print(f"Human template tokens: {human_tokens:.0f}")
print(f"Total tokens: {split_total_tokens:.0f}")
print(f"Input cost: ${split_input_cost:.6f}") 
print(f"Output cost: ${split_output_cost:.6f}")


🔧 Creating Split Prompt Agent...
✅ Split prompt agent: SplitPrompt_langchain_agent
📋 System message length: 3072
📋 Human template length: 1137

💰 Split Prompt Cost Estimates:
System tokens: 530
Human template tokens: 222
Total tokens: 753
Input cost: $0.000188
Output cost: $0.000941


In [4]:
# Test cases for comparison
test_cases = [
    "What is the copay for a doctor's visit?",
    "How do I apply for Medicaid?", 
    "Do I qualify for Medicare with my income?",
    "I need to know about Medicare eligibility and how to apply",
    "What are the requirements for CHIP, how do I apply, and what forms do I need?",
    "Help me understand my insurance benefits and fill out the enrollment form"
]

print("🧪 Testing Both Approaches")
print("=" * 60)

results = {"single": [], "split": []}

for i, test_case in enumerate(test_cases, 1):
    print(f"\n{i}. {test_case}")
    print("-" * 50)
    
    # Test single prompt approach
    try:
        single_result = single_prompt_agent(test_case)
        results["single"].append(single_result)
        print(f"Single: {single_result.workflows} | {single_result.priority} | {single_result.confidence}")
    except Exception as e:
        print(f"Single: ERROR - {str(e)[:50]}...")
        results["single"].append(None)
    
    # Test split prompt approach  
    try:
        split_result = split_prompt_agent(test_case)
        results["split"].append(split_result)
        print(f"Split:  {split_result.workflows} | {split_result.priority} | {split_result.confidence}")
    except Exception as e:
        print(f"Split:  ERROR - {str(e)[:50]}...")
        results["split"].append(None)

print(f"\n✅ Testing complete")


🧪 Testing Both Approaches

1. What is the copay for a doctor's visit?
--------------------------------------------------
Single: ['information_retrieval'] | low | 0.9
Split:  ['information_retrieval'] | low | 0.9

2. How do I apply for Medicaid?
--------------------------------------------------
Single: ['service_access_strategy'] | medium | 0.95
Split:  ['service_access_strategy'] | medium | 0.95

3. Do I qualify for Medicare with my income?
--------------------------------------------------
Single: ['determine_eligibility'] | medium | 0.9
Split:  ['determine_eligibility'] | medium | 0.9

4. I need to know about Medicare eligibility and how to apply
--------------------------------------------------
Single: ['determine_eligibility', 'service_access_strategy'] | medium | 0.85
Split:  ['determine_eligibility', 'service_access_strategy'] | medium | 0.85

5. What are the requirements for CHIP, how do I apply, and what forms do I need?
--------------------------------------------------
Sin

In [5]:
# Detailed comparison analysis
print("📊 Detailed Comparison Analysis")
print("=" * 60)

for i, test_case in enumerate(test_cases):
    single = results["single"][i]
    split = results["split"][i]
    
    print(f"\n{i+1}. {test_case[:50]}...")
    
    if single and split:
        print(f"   Single: {single.workflows}")
        print(f"   Split:  {split.workflows}")
        print(f"   Match:  {'✅' if single.workflows == split.workflows else '❌'}")
        print(f"   Confidence: Single={single.confidence}, Split={split.confidence}")
        print(f"   Priority: Single={single.priority}, Split={split.priority}")
    else:
        print(f"   Status: Single={'✅' if single else '❌'}, Split={'✅' if split else '❌'}")

# Summary statistics
single_success = sum(1 for r in results["single"] if r is not None)
split_success = sum(1 for r in results["split"] if r is not None)
matches = sum(1 for i in range(len(test_cases)) 
              if results["single"][i] and results["split"][i] 
              and results["single"][i].workflows == results["split"][i].workflows)

print(f"\n📈 Summary Statistics")
print(f"Single prompt success: {single_success}/{len(test_cases)}")
print(f"Split prompt success:  {split_success}/{len(test_cases)}")
print(f"Workflow matches:      {matches}/{min(single_success, split_success)}")

print(f"\n🏆 Comparison Results:")
if single_success > split_success:
    print("Single prompt approach had higher success rate")
elif split_success > single_success:
    print("Split prompt approach had higher success rate")
else:
    print("Both approaches had equal success rates")

if matches == min(single_success, split_success):
    print("All successful responses had matching workflows")
else:
    print(f"Only {matches} out of {min(single_success, split_success)} successful responses matched")


📊 Detailed Comparison Analysis

1. What is the copay for a doctor's visit?...
   Single: ['information_retrieval']
   Split:  ['information_retrieval']
   Match:  ✅
   Confidence: Single=0.9, Split=0.9
   Priority: Single=low, Split=low

2. How do I apply for Medicaid?...
   Single: ['service_access_strategy']
   Split:  ['service_access_strategy']
   Match:  ✅
   Confidence: Single=0.95, Split=0.95
   Priority: Single=medium, Split=medium

3. Do I qualify for Medicare with my income?...
   Single: ['determine_eligibility']
   Split:  ['determine_eligibility']
   Match:  ✅
   Confidence: Single=0.9, Split=0.9
   Priority: Single=medium, Split=medium

4. I need to know about Medicare eligibility and how ...
   Single: ['determine_eligibility', 'service_access_strategy']
   Split:  ['determine_eligibility', 'service_access_strategy']
   Match:  ✅
   Confidence: Single=0.85, Split=0.85
   Priority: Single=medium, Split=medium

5. What are the requirements for CHIP, how do I apply...
   Si